In [1]:
import numpy as np
import hail as hl
from hail import methods
import scipy as sp
import pandas as pd
from math import sqrt, pi
from os import path

def getIfNotExists(nsample = 1000, mvariants = 50_000):
    mtPath = f"balding_nichols_1_{nsample}_{mvariants}.mt"

    if not path.exists(mtPath):
        mt = hl.balding_nichols_model(1, nsample, mvariants)
        # this discards GT...wtf
        mt = hl.experimental.ldscsim.simulate_phenotypes(mt, mt.GT, .8)
        mt = mt.select_entries(ac = mt.GT.n_alt_alleles())
        mt.write(mtPath)
    else:
        mt = hl.methods.read_matrix_table(mtPath)
    
    return mt

mt = getIfNotExists(100, 20000)
print(mt.show())

mt.y.show()
mtOrig = mt

Initializing Hail with default parameters...
Running on Apache Spark version 2.4.1
SparkUI available at http://192.168.0.181:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.54-cbc3bbe7f14c
LOGGING: writing to /Users/alexkotlar/projects/hail/hail-20200821-1424-0.2.54-cbc3bbe7f14c.log


,,,,
,,0,1,2
locus,alleles,ac,ac,ac
locus<GRCh37>,array<str>,int32,int32,int32
1:1,"[""A"",""C""]",1,2,1
1:2,"[""A"",""C""]",1,2,1
1:3,"[""A"",""C""]",1,2,2
1:4,"[""A"",""C""]",2,1,1
1:5,"[""A"",""C""]",0,1,2
1:6,"[""A"",""C""]",0,2,1
1:7,"[""A"",""C""]",1,1,0


None


,
sample_idx,y
int32,float64
0,-3.66e-01
1,7.39e-02
2,-6.14e-01
3,-1.44e+00
4,1.78e+00
5,8.34e-01
6,1.20e+00
7,-1.24e-01


# Block genotype matrix, into nSample x kSnpsPerBlock x Bblocks

In [2]:
# TODO: support single-precision floats

import json
import numpy as np
from typing import Optional, Tuple, Callable

from hail.utils.java import Env
from hail.utils import range_table, new_temp_file
from hail.expr import Expression
from hail import nd
from hail.matrixtable import MatrixTable
from hail.table import Table
from hail.experimental.dnd import DNDArray
import hail as hl


mt = mtOrig
print("Original mt")
print(mt.show())
mt.describe()

n_variants_per_block = 1000
entrc_field = "ac"

n_rows, _ = mt.count()
n_block_rows = (n_rows + n_variants_per_block - 1) // n_variants_per_block
print(n_rows, n_rows, "n_block_rows", n_block_rows)

entries, cols, row_index, col_blocks = (Env.get_uid() for _ in range(4))


################### localize_entries takes the entry field, which is separated into n columns, and stuffs all of that data into a single field ##### 
# TODO: 
# Can do with an hl.agg.collect(ht[entrc_field]).rows instead of localize_entries
# localize entries does a copy to pull out an entry field

def genalphas(n_variants: int, r: int) -> np.array:
    h2 = np.arange(0.01, .99, 1/r, dtype=np.float32) 
    return n_variants / h2
    
alphas = genalphas(n_rows, 2)

alphaFields = [Env.get_uid() for _ in alphas]

y = np.array(mt.y.collect())
y = (y - y.mean()) / y.std()
ht = (mt
      .select_rows()
      .select_cols()
      .select_globals(Y = hl.nd.array(y),
                      # I = hl.nd.eye(n_variants_per_block),
                      alphas = alphas)
      .add_row_index(row_index)
      .localize_entries(entries))
print("""mt = (mt
      .add_row_index(row_index)
      .select_globals(Y = hl.nd.array(Y.collect()))
      .localize_entries(entries))""")
ht.describe()

sample_indices = mt.sample_idx.collect()


################## This step transformed the array of structs to an array of the values of one of the fields in that struct ########

ht = ht.annotate(**{entries: ht[entries][entrc_field]})
print("after mt.annotate(**{entries: mt[entries][entrc_field]})")
ht.describe()

ht = ht.annotate(xmean = hl.mean(ht[entries]))
ht = ht.annotate(xstd = (hl.sum(ht[entries].map(lambda x: (x - ht.xmean) ** 2)) / hl.len(ht[entries])) ** .5)
ht = ht.annotate(X = ht[entries].map(lambda x: (x - ht.xmean) / ht.xstd))

################ Group rows of snps (which are already collected into nSample x 1 arrays) ######### 
ht = ht.annotate(r=hl.int(ht[row_index] // n_variants_per_block))
print("after mt = mt.annotate(r=hl.int(mt[row_index] // n_variants_per_block))")
print(ht.show())
ht.describe()

# Not sure of the benefit of key by outside of joins...does this create an index? does it sort? if it sorts, that is magic
# does sort; mostly for joins, but if something doesn't use that sort order we are allowed
# implicit join syntax
# because we can store on disk ordered by key
ht = ht.key_by(ht.r)
print("after mt = mt.key_by(mt.r, mt.c)")
print(ht.show())
ht.describe()

# Not sure why hl.sorted is neededd here
# TODO: I think this should be group by chromosome and row block
ht = ht.group_by(ht.r).aggregate(
    entries=hl.sorted(
        hl.agg.collect(hl.struct(row_index=ht[row_index], X=ht.X)),
        key=lambda x: x.row_index
    ))
print("""after ht = ht.group_by(ht.r).aggregate(
    entries=hl.sorted(
        hl.agg.collect(hl.struct(row_index=ht[row_index], X=ht.X)),
        key=lambda x: x.row_index
    ))""")
ht.describe()

# X will now be shape nSample x bSnps
# so XtX will be bSnps x bSnps
ht = ht.select(X = hl.nd.array(ht.entries.X).T)

print("after mt = mt.select(block=hl.nd.array(mt.entries))")
ht.describe()

ht = ht.select_globals(
    Y = ht.Y,
    alphas = ht.alphas,
    alphaFields = alphaFields,
    snp_block_idx_field='r',
    n_variants_per_block=n_variants_per_block,
    n_block_rows=n_block_rows)
print("""after ht = ht.select_globals(
    Y = ht.Y,
    alphas = ht.alphas,
    alphaFields = alphaFields,
    snp_block_idx_field='r',
    n_variants_per_block=n_variants_per_block,
    n_block_rows=n_block_rows)""")
ht.describe()

Original mt


,,,,
,,0,1,2
locus,alleles,ac,ac,ac
locus<GRCh37>,array<str>,int32,int32,int32
1:1,"[""A"",""C""]",1,2,1
1:2,"[""A"",""C""]",1,2,1
1:3,"[""A"",""C""]",1,2,2
1:4,"[""A"",""C""]",2,1,1
1:5,"[""A"",""C""]",0,1,2
1:6,"[""A"",""C""]",0,2,1
1:7,"[""A"",""C""]",1,1,0


None
----------------------------------------
Global fields:
    'bn': struct {
        n_populations: int32, 
        n_samples: int32, 
        n_variants: int32, 
        n_partitions: int32, 
        pop_dist: array<int32>, 
        fst: array<float64>, 
        mixture: bool
    }
    'ldscsim': struct {
        h2: float64, 
        exact_h2: bool
    }
----------------------------------------
Column fields:
    'sample_idx': int32
    'pop': int32
    'y_no_noise': float64
    'y': float64
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'ancestral_af': float64
    'af': array<float64>
    'beta': float64
----------------------------------------
Entry fields:
    'ac': int32
----------------------------------------
Column key: ['sample_idx']
Row key: ['locus', 'alleles']
----------------------------------------
19914 19914 n_block_rows 20
mt = (mt
      .add_row_index(row_index)
      .select_globals(Y = hl.nd.array(Y.

,,,,,,,
locus,alleles,__uid_9,__uid_7,xmean,xstd,X,r
locus<GRCh37>,array<str>,int64,array<int32>,float64,float64,array<float64>,int32
1:1,"[""A"",""C""]",0,"[1,2,1,1,0,0,2,2,1,2,1,0,2,1,1,1,1,1,2,2,1,2,2,2,0,1,1,1,1,1,1,0,1,2,2,1,2,1,0,2,2,1,2,0,2,0,0,2,1,1,2,1,1,2,1,2,1,1,2,1,0,1,1,1,1,2,1,1,2,1,2,1,1,2,2,2,1,2,2,1,2,2,1,2,2,1,1,1,1,2,2,1,2,2,1,1,1,2,2,0]",1.29e+00,6.53e-01,"[-4.44e-01,1.09e+00,-4.44e-01,-4.44e-01,-1.98e+00,-1.98e+00,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,-4.44e-01,-1.98e+00,1.09e+00,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,1.09e+00,1.09e+00,-1.98e+00,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,-1.98e+00,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,-4.44e-01,-1.98e+00,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,-1.98e+00,1.09e+00,-1.98e+00,-1.98e+00,1.09e+00,-4.44e-01,-4.44e-01,1.09e+00,-4.44e-01,-4.44e-01,1.09e+00,-4.44e-01,1.09e+00,-4.44e-01,-4.44e-01,1.09e+00,-4.44e-01,-1.98e+00,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,1.09e+00,-4.44e-01,-4.44e-01,1.09e+00,-4.44e-01,1.09e+00,-4.44e-01,-4.44e-01,1.09e+00,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,-4.44e-01,-4.44e-01,1.09e+00,1.09e+00,-1.98e+00]",0
1:2,"[""A"",""C""]",1,"[1,2,1,0,2,2,2,1,1,2,1,1,1,2,2,2,1,0,1,2,2,1,1,1,2,1,2,0,0,1,2,1,1,1,1,2,0,2,1,1,1,1,1,1,1,1,2,0,1,2,0,1,0,1,1,0,2,1,1,1,1,1,1,2,1,2,2,0,1,1,2,1,2,1,2,2,0,1,1,1,0,2,1,2,1,1,1,1,1,0,1,1,1,1,0,2,1,1,0,1]",1.13e+00,6.43e-01,"[-2.02e-01,1.35e+00,-2.02e-01,-1.76e+00,1.35e+00,1.35e+00,1.35e+00,-2.02e-01,-2.02e-01,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,1.35e+00,1.35e+00,1.35e+00,-2.02e-01,-1.76e+00,-2.02e-01,1.35e+00,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,1.35e+00,-2.02e-01,1.35e+00,-1.76e+00,-1.76e+00,-2.02e-01,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,1.35e+00,-1.76e+00,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,1.35e+00,-1.76e+00,-2.02e-01,1.35e+00,-1.76e+00,-2.02e-01,-1.76e+00,-2.02e-01,-2.02e-01,-1.76e+00,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,1.35e+00,-2.02e-01,1.35e+00,1.35e+00,-1.76e+00,-2.02e-01,-2.02e-01,1.35e+00,-2.02e-01,1.35e+00,-2.02e-01,1.35e+00,1.35e+00,-1.76e+00,-2.02e-01,-2.02e-01,-2.02e-01,-1.76e+00,1.35e+00,-2.02e-01,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-1.76e+00,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-1.76e+00,1.35e+00,-2.02e-01,-2.02e-01,-1.76e+00,-2.02e-01]",0
1:3,"[""A"",""C""]",2,"[1,2,2,0,2,2,1,2,1,2,2,1,1,2,1,2,2,2,1,2,0,2,2,2,1,1,2,2,2,1,2,0,1,1,0,1,2,2,0,1,2,1,1,2,2,1,2,2,2,2,1,1,2,1,2,1,2,2,1,1,2,1,2,1,1,2,1,2,2,2,1,1,1,0,1,2,0,1,1,1,2,1,1,2,0,0,0,1,2,1,2,2,2,1,2,2,2,1,2,0]",1.38e+00,6.75e-01,"[-5.63e-01,9.19e-01,9.19e-01,-2.04e+00,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,-2.04e+00,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,-2.04e+00,-5.63e-01,-5.63e-01,-2.04e+00,-5.63e-01,9.19e-01,9.19e-01,-2.04e+00,-5.63e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,-5.63e-01,-5.63e-01,-2.04e+00,-5.63e-01,9.19e-01,-2.04e+00,-5.63e-01,-5.63e-01,-5.63e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,-2.04e+00,-2.04e+00,-2.04e+00,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,-2.04e+00]",0
1:4,"[""A"",""C""]",3,"[2,1,1,1,1,2,1,2,2,1,0,2,1,1,1,1,2,2,1,2,1,2,2,1,2,2,2,1,1,2,2,2,2,1,1,2,1,0,1,1,1,2,2,2,0,0,2,2,1,1,2,1,1,1,1,1,1,2,1,0,1,1,0,1,2,2,2,1,2,1,1,2,1,2,1,1,2,1,1,1,0,1,1,1,2,2,0,0,0,2,2,1,1,1,2,2,2,1,2,0]",1.29e+0

None
----------------------------------------
Global fields:
    'Y': ndarray<float64, 1> 
    'alphas': ndarray<float32, 1> 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    '__uid_9': int64 
    '__uid_7': array<int32> 
    'xmean': float64 
    'xstd': float64 
    'X': array<float64> 
    'r': int32 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------
after mt = mt.key_by(mt.r, mt.c)


,,,,,,,
locus,alleles,__uid_9,__uid_7,xmean,xstd,X,r
locus<GRCh37>,array<str>,int64,array<int32>,float64,float64,array<float64>,int32
1:1,"[""A"",""C""]",0,"[1,2,1,1,0,0,2,2,1,2,1,0,2,1,1,1,1,1,2,2,1,2,2,2,0,1,1,1,1,1,1,0,1,2,2,1,2,1,0,2,2,1,2,0,2,0,0,2,1,1,2,1,1,2,1,2,1,1,2,1,0,1,1,1,1,2,1,1,2,1,2,1,1,2,2,2,1,2,2,1,2,2,1,2,2,1,1,1,1,2,2,1,2,2,1,1,1,2,2,0]",1.29e+00,6.53e-01,"[-4.44e-01,1.09e+00,-4.44e-01,-4.44e-01,-1.98e+00,-1.98e+00,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,-4.44e-01,-1.98e+00,1.09e+00,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,1.09e+00,1.09e+00,-1.98e+00,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,-1.98e+00,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,-4.44e-01,-1.98e+00,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,-1.98e+00,1.09e+00,-1.98e+00,-1.98e+00,1.09e+00,-4.44e-01,-4.44e-01,1.09e+00,-4.44e-01,-4.44e-01,1.09e+00,-4.44e-01,1.09e+00,-4.44e-01,-4.44e-01,1.09e+00,-4.44e-01,-1.98e+00,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,1.09e+00,-4.44e-01,-4.44e-01,1.09e+00,-4.44e-01,1.09e+00,-4.44e-01,-4.44e-01,1.09e+00,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,-4.44e-01,-4.44e-01,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,1.09e+00,1.09e+00,-4.44e-01,-4.44e-01,-4.44e-01,1.09e+00,1.09e+00,-1.98e+00]",0
1:2,"[""A"",""C""]",1,"[1,2,1,0,2,2,2,1,1,2,1,1,1,2,2,2,1,0,1,2,2,1,1,1,2,1,2,0,0,1,2,1,1,1,1,2,0,2,1,1,1,1,1,1,1,1,2,0,1,2,0,1,0,1,1,0,2,1,1,1,1,1,1,2,1,2,2,0,1,1,2,1,2,1,2,2,0,1,1,1,0,2,1,2,1,1,1,1,1,0,1,1,1,1,0,2,1,1,0,1]",1.13e+00,6.43e-01,"[-2.02e-01,1.35e+00,-2.02e-01,-1.76e+00,1.35e+00,1.35e+00,1.35e+00,-2.02e-01,-2.02e-01,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,1.35e+00,1.35e+00,1.35e+00,-2.02e-01,-1.76e+00,-2.02e-01,1.35e+00,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,1.35e+00,-2.02e-01,1.35e+00,-1.76e+00,-1.76e+00,-2.02e-01,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,1.35e+00,-1.76e+00,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,1.35e+00,-1.76e+00,-2.02e-01,1.35e+00,-1.76e+00,-2.02e-01,-1.76e+00,-2.02e-01,-2.02e-01,-1.76e+00,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,1.35e+00,-2.02e-01,1.35e+00,1.35e+00,-1.76e+00,-2.02e-01,-2.02e-01,1.35e+00,-2.02e-01,1.35e+00,-2.02e-01,1.35e+00,1.35e+00,-1.76e+00,-2.02e-01,-2.02e-01,-2.02e-01,-1.76e+00,1.35e+00,-2.02e-01,1.35e+00,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-1.76e+00,-2.02e-01,-2.02e-01,-2.02e-01,-2.02e-01,-1.76e+00,1.35e+00,-2.02e-01,-2.02e-01,-1.76e+00,-2.02e-01]",0
1:3,"[""A"",""C""]",2,"[1,2,2,0,2,2,1,2,1,2,2,1,1,2,1,2,2,2,1,2,0,2,2,2,1,1,2,2,2,1,2,0,1,1,0,1,2,2,0,1,2,1,1,2,2,1,2,2,2,2,1,1,2,1,2,1,2,2,1,1,2,1,2,1,1,2,1,2,2,2,1,1,1,0,1,2,0,1,1,1,2,1,1,2,0,0,0,1,2,1,2,2,2,1,2,2,2,1,2,0]",1.38e+00,6.75e-01,"[-5.63e-01,9.19e-01,9.19e-01,-2.04e+00,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,-2.04e+00,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,-2.04e+00,-5.63e-01,-5.63e-01,-2.04e+00,-5.63e-01,9.19e-01,9.19e-01,-2.04e+00,-5.63e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,-5.63e-01,-5.63e-01,-2.04e+00,-5.63e-01,9.19e-01,-2.04e+00,-5.63e-01,-5.63e-01,-5.63e-01,9.19e-01,-5.63e-01,-5.63e-01,9.19e-01,-2.04e+00,-2.04e+00,-2.04e+00,-5.63e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,9.19e-01,9.19e-01,-5.63e-01,9.19e-01,-2.04e+00]",0
1:4,"[""A"",""C""]",3,"[2,1,1,1,1,2,1,2,2,1,0,2,1,1,1,1,2,2,1,2,1,2,2,1,2,2,2,1,1,2,2,2,2,1,1,2,1,0,1,1,1,2,2,2,0,0,2,2,1,1,2,1,1,1,1,1,1,2,1,0,1,1,0,1,2,2,2,1,2,1,1,2,1,2,1,1,2,1,1,1,0,1,1,1,2,2,0,0,0,2,2,1,1,1,2,2,2,1,2,0]",1.29e+0

None
----------------------------------------
Global fields:
    'Y': ndarray<float64, 1> 
    'alphas': ndarray<float32, 1> 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    '__uid_9': int64 
    '__uid_7': array<int32> 
    'xmean': float64 
    'xstd': float64 
    'X': array<float64> 
    'r': int32 
----------------------------------------
Key: ['r']
----------------------------------------
after ht = ht.group_by(ht.r).aggregate(
    entries=hl.sorted(
        hl.agg.collect(hl.struct(row_index=ht[row_index], X=ht.X)),
        key=lambda x: x.row_index
    ))
----------------------------------------
Global fields:
    'Y': ndarray<float64, 1> 
    'alphas': ndarray<float32, 1> 
----------------------------------------
Row fields:
    'r': int32 
    'entries': array<struct {
        row_index: int64, 
        X: array<float64>
    }> 
----------------------------------------
Key: ['r']
---------------------------------

In [ ]:
# y = np.array(mt.y.collect())
# y = (y - y.mean()) / y.std()
# # Can do with an hl.agg.collect(ht[entrc_field]).rows instead of localize_entries
# # localize entries does a copy to pull out an entry field
# ht = (mt
#       .select_rows()
#       .select_cols()
#       .select_globals(Y = hl.nd.array(y),
# #                       I = hl.nd.eye(n_variants_per_block),
#                       alphas = alphas)
#       .add_row_index(row_index)
#       .localize_entries(entries))



In [3]:
x = ht.X.take(1)[0]
xn = np.array(x)
assert(np.allclose(xn.std(0), 1))
assert(np.allclose(xn.mean(0), 0))

2020-08-21 14:27:12 Hail: INFO: Ordering unsorted dataset with network shuffle
(100, 1000)


In [4]:
# TODO: store I in globals to prevent re-computation, will need to account for the last block being improperly sized
ht2 = ht.annotate(XtX = ht.X.T @ ht.X, XtY = ht.X.T @ ht.Y, I = hl.nd.eye(ht.X.shape[1]))

In [5]:
ht2.describe()

----------------------------------------
Global fields:
    'Y': ndarray<float64, 1> 
    'alphas': ndarray<float32, 1> 
    'alphaFields': array<str> 
    'snp_block_idx_field': str 
    'n_variants_per_block': int32 
    'n_block_rows': int32 
----------------------------------------
Row fields:
    'r': int32 
    'X': ndarray<float64, 2> 
    'XtX': ndarray<float64, 2> 
    'XtY': ndarray<float64, 1> 
    'I': ndarray<float64, 2> 
----------------------------------------
Key: ['r']
----------------------------------------


In [ ]:
# # ################## This is insaneballs slow, like straight up hang forever ################
# ht2 = ht2.annotate(B = hl.nd.array(ht2.alphas.map(lambda alpha: (
#     (hl.nd.inv(ht2.XtX + alpha * ht2.I) @ ht2.XtY)._data_array()
# ))))

# ht2 = ht2.annotate(W = ht2.block.T @ ht2.B)

In [ ]:
# 2nd attempt, can't fucking stack because "row not found: {}" from line 390 in compute_type
# exprs = {}
# for i in range(len(alphas)):
#     exprs[alphaFields[i]] = hl.nd.inv(ht2.XtX + ht2.alphas[i] * ht2.I) @ ht2.XtY
# print(exprs)
# # ht2 = ht2.annotate(**exprs)
# ht2 = ht2.annotate(B = hl.nd.vstack([ht2[field] for field in alphaFields]))

In [6]:
################# A version that tries to avoid the performance cliff above ###############
# Equation 14 in https://www.biorxiv.org/content/10.1101/2020.06.19.162354v2.full.pdf
# 􏰉λ =(GTi Gi +λIMi)−1GTi y􏰊, λ∈Λ
exprs = {}
alphaFields = ht2.alphaFields.collect()[0]
print(alphaFields)
for i in range(len(alphaFields)):
    exprs[alphaFields[i]] = hl.nd.inv(ht2.XtX + ht2.alphas[i] * ht2.I) @ ht2.XtY
ht2 = ht2.annotate(**exprs)
ht2 = ht2.annotate(B = [ht2[field]._data_array() for field in alphaFields])

# B is shape (2, 1000); 2 rows of 1000 columns, so k x b where b is number of snps in a block
# so we transpose to make b x k
ht2 = ht2.annotate(B = hl.nd.array(ht2.B).T)


['__uid_11', '__uid_12']


In [7]:
# Equation 15
# Wi = (Gi * γ􏰉_λ1,...,Gi * γ􏰉_λR), i = 1,...,B
# G is n x b, y_i is b x 1
# B is now b x k, so n x b @ b x k gives us n x k
ht2 = ht2.annotate(Wi = ht2.X @ ht2.B)
ht2.describe()

# We now need to standardize Wi
# Wi is n x k shape
ht2 = ht2.annotate(WiMean = hl.nd.array(hl.range(len(alphas)).map(lambda kIdx: ( hl.sum(ht2.Wi[:, kIdx]._data_array())) / ht2.Wi[:, kIdx].shape[0])))
# would we ever standardize using sample standard deviation (n-1)?
ht2 = ht2.annotate(WiSD = hl.nd.array(hl.range(len(alphas)).map(lambda kIdx: ( hl.sum(ht2.Wi[:,kIdx]._data_array().map(lambda el: (el - ht2.WiMean[kIdx]) ** 2)) / (ht2.Wi[:, kIdx].shape[0])) ** .5 )))
ht2 = ht2.annotate(WiNormalized = (ht2.Wi - ht2.WiMean) / ht2.WiSD)



----------------------------------------
Global fields:
    'Y': ndarray<float64, 1> 
    'alphas': ndarray<float32, 1> 
    'alphaFields': array<str> 
    'snp_block_idx_field': str 
    'n_variants_per_block': int32 
    'n_block_rows': int32 
----------------------------------------
Row fields:
    'r': int32 
    'X': ndarray<float64, 2> 
    'XtX': ndarray<float64, 2> 
    'XtY': ndarray<float64, 1> 
    'I': ndarray<float64, 2> 
    '__uid_11': ndarray<float64, 1> 
    '__uid_12': ndarray<float64, 1> 
    'B': ndarray<float64, 2> 
    'Wi': ndarray<float64, 2> 
----------------------------------------
Key: ['r']
----------------------------------------


In [8]:
ht2 = ht2.select("WiNormalized")

In [9]:
ht2 = ht2.checkpoint('tmp/table_checkpoint.ht', overwrite = True)

# ht2 = hl.read_table('tmp/table_checkpoint.ht')
# ht2.show(1)

2020-08-21 14:28:07 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-08-21 14:28:18 Hail: INFO: wrote table with 20 rows in 8 partitions to tmp/table_checkpoint.ht


In [ ]:
####### Level 1
# Now we want to:
# 1) create an array W, which. is nSample x b * B (b snps per block * B blocks) matrix
# 2) split this matrix into k folds (maybe randomly?), row-wise ( create blcoks of samples, each with bB columns)
# 3) take random k - 1 groups for train data, 1 block for test data
# 4) test 5-10 alphas per block, performing ridge regression
# 5) choose the winning alpha from each block test by examing the out-of-fold loss (you do this by taking the sum of squared error in the test data)
# 6) chooset the winning alpha across blocks
# TODO: double check this impression

# Definitions: https://stackoverflow.com/questions/19335165/what-is-the-difference-between-cross-validation-and-grid-search
# It's when you hold out some samples to test on, train on the rest
# Grid search definition:
# Systematically check all parameter combinations, and take the best cross-validation result
# We need to do a grid search over parameters lambda and k
# As a first step, we can simply do a grid search over lambda
# Take the first k values as test data, and the rest as test data




# Now I believe we want to block this array in nSample/kFolds x KBs pieces, and choice random collections of k row groups, 1 for test for training data (or more simply 0:k and 1:k sample groups 
# if hail can't easily handle the pemutations)
# So here we're partitioning by rows
# TODO: Could this be done in TableMapRows or TableMapPartitions? From Cotton's description this goes from ValueIR -> TableIR
# TODO: double check whether WGR or regenie does random permutations



# Fuck the regenie paper for making the concept so confusing, god...academics
# https://www.kdnuggets.com/2017/02/stacking-models-imropved-predictions.html
# "Each model provides predictions for the outcome (y) which are then cast into a second 
# level training data (Xl2) which is now m x M. Namely, the M predictions become features
# for this second level data."
# The point is the the "J" dimension from the last step becomes our feature dimension
# so we went from n x b to n x j, where j is 2 or 5 or 10, 1 for each alpha value (and b was 1000, one for each snp in the block)
# so as long as J is less than the block size, we're good. Cool!
# Then we just redo the step above, again.

# And the reason they want to do K-fold CV, is:
# "The key word here is out-of-sample, since if we were to use predictions from the M models that are fit to all the training data, then the second level model
#will be biased towards the best of M models. This will be of no use."

# Looking at https://github.com/projectglow/glow/blob/054c3f4c714cf1693c964554842bf3ba67d76eb4/levels_ridge_regression_tutorial.ipynb
# For this next step, we generate a different set of alphas (although this isn't critical accoridng to Kaggle)
# Unclear: do we join all blocks at this point?
# I think we join https://github.com/projectglow/glow/blob/master/python/glow/wgr/linear_model/ridge_model.py#L216

# TODO: this may need to be group by chromsoome
# ht = ht.group_by(ht.r).aggregate(
#     entries=hl.sorted(
#         hl.agg.collect(hl.struct(row_index=ht[row_index], X=ht.X)),
#         key=lambda x: x.row_index
#     ))
# For now:
# I may need to concatenate these blocks together, stacking them is not right, but maybe
# reshape can get what I need
# ht2 = ht2.annotate(g = 0)
# ht3 = ht2.group_by(ht2.g).aggregate(W = hl.agg.collect(ht2.WiNormalized))

# ht3.describe()
# ht3.W.collect()
# I mean, i'm not really sure at the relative merits of keeping this all in Table. Seems easy enough, small enough to just grab
# so we have what, maximally 1 million samples by 10-100 predictors, 8 bytes? Like 1GB. No biggie, altohugh latency on operation will
# be terrible if on some slow local network, and totally ok, if this is being perfomed
# in a VPC
# Ah no, we've not blocked on samples. We took all variants
# ah, but WGR is still blocking on samples, but using all variants. Hm, not sure
# if we should continue blockign on samples or collecting
# Collecting is easier, so let's just do that.

# TODO: get the table version up
# ndstack = hl.nd.hstack(nd)
# # print("ndstack", hl.eval(ndstack).shape)
# # assert(np.array_equal(np.hstack(nd), hl.eval(ndstack)))

# # TODO: in the future g should be a chromosome grouping
# # TODO: what is the performance impaact of agg.collect() vs W = ht2.WiNormalized.collect()
# ht3 = ht2.annotate(g = 0)
# # # ._data_array() because we can't make ndarrays from array<ndarray<>> yet
# ht3 = ht3.group_by(ht3.g).aggregate(W = hl.agg.collect(ht3.WiNormalized))
# ht3 = ht3.annotate(Wstack = hl.nd.hstack(ht3.W))


# ht3.describe()

########################  Numpy version ##########################
# We can now hstack in hail
W = ht2.WiNormalized.collect()

# Now we have a nSamples x kFeatures*Bblock matrix
W = np.hstack(W)
W.shape



In [ ]:
# Our full version will probably block on sample
ht3 = ht2.annotate(g = 0)
# # ._data_array() because we can't make ndarrays from array<ndarray<>> yet
# W is an array of len 20, because we have 20 blocks (20_000 snps / 1_000 snps per block)
# Each represents all samples, and 1/20th the features
# we need to get the features into one
# ht3 = ht3.group_by(ht3.g).aggregate(W = hl.agg.collect(ht3.WiNormalized))
# ht3 = ht3.select(Wstack = hl.nd.hstack(ht3.W))
# We really only need to flatten the top level


ht3.show()
ht3.describe() 


In [ ]:
ht3._map_partitions(lambda rows: rows.flatmap(lambda r: hl.range(2).map(lambda x: r.annotate(f = r.g)))).show()

In [ ]:
from hail.ir import Apply, TableMapRows, MatrixKeyRowsBy, TopLevelReference


In [ ]:
htt = hl.utils.range_table(2)
htt2 = htt._map_partitions(lambda rows: rows.flatmap(lambda r: hl.range(2).map(lambda x: r.annotate(x=x))))
htt2.show()

In [ ]:
# There is now 1 array per sample
# We should just have explode on ndarray
ht4 = ht3.select(W2 = ht3.Wstack._data_array())
ht4 = ht4.explode(ht4.W2)
ht4 = ht4.annotate(W2 = hl.nd.array(ht4.W2))

# Only need if we explode
# this could be done as a first step, with row_index, and kept instead of wasting work here
ht4 = ht4.add_index("sidx")

In [ ]:
ht4.describe()
# slow as shit: ht4.to_matrix_table(["g"], ["sidx"]).show()

In [ ]:
###### K fold cross validation
# TODO: we'll need to make sure the sidx match the Y global
# For k folds , the experiment is repeate k times
k_folds = 5
test_prob = 1 / k_folds
# 5 folds of cross validation
# could 
exprs = {}
for fold in range(k_folds):
    exprs[f"fold_{fold}"] = hl.rand_bool(test_prob, seed=fold)

ht4 = ht4.annotate(**exprs)
ht4.show()

In [ ]:
ht4.row

In [ ]:
ht5 = ht4.group_by(ht4.g) \
         .aggregate(train_test = hl.agg.group_by(hl.rand_bool(test_prob, seed=fold), hl.agg.collect(ht4.row)))
ht5.describe()

In [ ]:
ht5.show()

In [ ]:
keys = exprs.keys()
keys
ht5 = ht4.annotate(Wtrain = ht4.aggregate(hl.agg.filter(ht4["fold_0"] == True, hl.agg.collect(ht4.W2))))
# hl.array([value for key, value in ht4.row_value.items()])

In [ ]:
hl.utils.range_table(5).annotate(x = ht3.Wstack[0:5, :])

In [ ]:
ht6 = ht5.annotate

In [ ]:
ht5 = ht4.group_by(*key).aggregate(
    **{fv: hl.agg.filter(ht[field] == fv,
        hl.rbind(
                                                hl.agg.take(ht[value], 1),
                                                lambda take: hl.cond(hl.len(take) > 0,
                                                                     take[0],
                                                                     'NA'))) for fv in field_vals})

In [ ]:
# ht5 = ht4.annotate(WTrain = ht4.filter(ht4["fold_0"] == True).group_by("fold_0")).aggregate(hl.agg.collect(ht4.W2))

In [ ]:
ht4.idx

In [ ]:
# TODO: can we avoid storing the bigger fold of indices?
# Also, sidx does not need to be 64 bit int
# for key in exprs.keys():
#     ht5 = ht4.group_by(key).aggregate(WFold = hl.nd.vstack(hl.agg.collect(ht4.W2)), sidx = hl.agg.collect(ht4.sidx))
    
#     ht5 = ht5.WFold.map(lambda rows: rows.map(lambda r: hl.if_else(r.idx == 0, r.annotate(x = 5), r.annotate(x =6))))
#     ht5.show()
#     break

# ht5.show()
#ht5 == 0
# trainRow = ht5[0]
# testRow =  ht5[ht5["fold_0"] = 1]
ht5 = ht5.annotate(WtW = ht5.WFold.T @ ht5.WFold)
# Wtrain = trainRow.WFold
# Wtest = testRow.WFold
# sidxTrain = trainRow.sidx
# sidxTest = testRow.sidx

# Ytrain = sidxTrain.map(lambda i: ht5.Y[i])
# Ytest = sidxTest.map(lambda i: ht5.Y[i])

# Itrain = hl.nd.eye(Wtrain.shape[0])
# Itest = hl.nd.eye(Wtest.shape[0])

# bestIdx = 0

# # This doesn't work as expected
# WtWtrain = Wtrain.T @ Wtrain
# WtYtrain = Wtrain.T @ Ytrain
# WtWtest = Wtest.T @ Wtest
# WtYtest = Wtest.T @ Ytest

# WtWtrain.show()
# res = hl.nd.inv(WtWtrain)
# # losses = []

# trainRow = trainRow.annotate(fuckyou = 1)
# # for alpha in alphas:
# #     BetaHat = hl.nd.inv(WtWtrain + alpha * Itrain) @ WtYtrain
# #     ht6 = ht5.annotate(BetaHat = BetaHat)

# ht6 = ht5[ht5["fold_0"] == True].annotate(WFold = 5)

In [ ]:
### Overall plan
# Starting point is nSample x rb matrices
# In current iteration nSample is 1, after an explode
# generate K splits of 

In [ ]:
# Item assignment doesn't work, wtf
# ht5[ht5["fold_0"] == True] = ht5[ht5["fold_0"] == True].annotate(WFold = 5)

ht5.describe()

In [ ]:
ht5.show()


In [ ]:
ht6 = ht5._map_partitions(lambda rows: rows.flatmap(lambda r: hl.range(2).map(lambda x: r.annotate(x=x))))

In [ ]:
hl.eval(hl.struct())

In [ ]:
ht6.show()

In [ ]:
test.collect()[0].shape

In [ ]:
ht5.Y.collect()[0].shape

In [ ]:
# h5 = ht4.group_by(is_test).aggregate()

In [ ]:
ht = hl.utils.range_table(2)
ht2 = ht._map_partitions(lambda rows: rows.flatmap(lambda r: hl.range(2).map(lambda x: r.annotate(x=x))))
ht2.show()
ht.show()

# This allows us to set conditional values 
ht3 = ht._map_partitions(lambda rows: rows.map(lambda r: hl.if_else(r.idx == 0, r.annotate(x = 5), r.annotate(x =6))))
ht3.show()

# does this do the same?
# nope
ht4 = ht.idx.map(lambda rows: rows.map(lambda r: hl.if_else(r.idx == 0, r.annotate(x = 5), r.annotate(x =6))))
ht4.show()

In [ ]:
ht4.show()

In [ ]:
# mt = ht3.to_matrix_table("[ht3.g]", ht3.sidx)

In [ ]:
mt = ht3.to_matrix_table

In [ ]:
ht3.show()

In [ ]:
ht3.Wstack.take(1)[0].shape

In [ ]:
ht = hl.utils.range_table(10)
ht

ht = ht.annotate(x=hl.nd.array([1, 2, 3]), y=hl.nd.array([4, 5, 6]))
ht
ht = ht.annotate(stacked = hl.nd.hstack([ht.x, ht.y]))
assert np.array_equal(ht.collect()[0].stacked, np.array([1, 2, 3, 4, 5, 6]))


In [ ]:
ht2.annotate(alphasToTest = alphas).show(2)

In [ ]:
# Scikit learn blocking
k_folds= 5
test_size = 1 / k_folds

# from sklearn import cross_validation
from sklearn.model_selection import train_test_split

for alpha in alphas:
    W_train, W_test, y_train, y_test = train_test_split(W, y, test_size=test_size, random_state=0)
    print(W_train)

In [ ]:
alphas

In [ ]:
W_train.shape

In [ ]:
ht3.explode("W").describe()

In [ ]:
np.array(ht3.W.take(1)[0]).shape

In [ ]:
ht3.describe()

In [ ]:
mt3 = ht3.to_matrix_table_row_major(["W"], entry_field_name="h")
mt3.show()

In [ ]:
## Testing slicing functionality in hail, to see if we can achieve what we desire

In [ ]:
ht3 = ht3.annotate(Wnd = hl.nd.array(ht3.W))

np.array(ht3.Wnd.take(1)[0]).shape

In [ ]:
import time
start = time.time()
ht3.count()
print(time.time() - start)

In [ ]:
# mt = = ht3.annotate(W = hl.nd.array(ht3.W))
# We cannot slice using a list!!!
# Damnit
# In [18]: a[[0,1,2,3,4,8]]                                                                                                                              
# Out[18]: array([ 1,  2,  3,  5,  6, 10])

# In [19]: ah = hl.nd.array(a)                                                                                                                           
# ---------------------------------------------------------------------------
# NameError                                 Traceback (most recent call last)
# <ipython-input-19-5ee9daf5500e> in <module>
# ----> 1 ah = hl.nd.array(a)

# NameError: name 'hl' is not defined

# In [20]: import hail as hl                                                                                                                             

# In [21]: ah = hl.nd.array(a)                                                                                                                           

# In [22]: hl.eval(ah[[1,2,3,9,10]])  


### Remaining questions: is collecting all chunks into one array within a table expression
# better than collect() locally? probably if I split afterwards?
# 2nd: woudl it be more efficient to not collect at all? no, that seems crazy, would require K aggregations
### First attempt: do this all in Table, and use stupid, 0:(K-1)*chunk_size + 1 (k-1) * chunk_size: end splits
n_samples = y.shape[0]
kfolds = 5
train_folds = kfolds - 1
test_folds = 1
row_chunk_size = n_samples // kfolds

# TODO: I think another option is to explode W (before making )
# TODO: obviously I shouldn't split into 0:train and train:0, they're identical, just labels flipped
ht3 = ht3.annotate(W_train = ht3.W[0:row_chunk_size*train_folds + 1])
ht3 = ht3.annotate(W_test= ht3.W[row_chunk_size*train_folds + 1:])
ht3 = ht3.annotate(W_test_1 = ht3.W[0:row_chunk_size*train_folds + 1])
ht3 = ht3.annotate(W_train_1 = ht3.W[row_chunk_size*train_folds + 1:])

ht3.show()
ht3.W_test_1.take(1)
    
    # we split the data into 2 groups n_samples 
    

In [ ]:
# TODO: Now I need to split this dataset into a bunch of rows I think...this will surely help hail map the things in a distributed fashion



In [ ]:
## In the next step, we block on samples, not on predictors
# We block using a K-fold strategy
# Since our array here is small, may as well do it in numpy, can translate to Hail later

In [ ]:
wi = ht2.Wi.take(1)[0]
wimean = ht2.WiMean.take(1)[0]
wisd = ht2.WiSD.take(1)[0]
assert(np.allclose(wimean, wi.mean(0)))
assert(np.allclose(wisd, wi.std(0)))

print("shape", wi.shape, "sd", wisd, "mean", wimean)


In [ ]:
wi.std(0)

In [ ]:
# print(row_index)
# Not sure why entries=hl.sorted is neededd here
# is hl.sorted needed here?
# W is now shape (nBlocks, nSamples, kPredictors)
# for 500,00 samples, and 1 million variants, and a 1000 block size, and 5 predictors
# this would be (1_000, 500_000, 5)
# paper claims that this would be smaller than the full genotype matrix... is it?
# Sure, 1_000 by 500_000 is smaller than 500_000 by 500_000, but just storing Betas would be smaller still
# 1_000 by 1_000 by 5...

# print(ht3.show())
# ht3.

# ht3 = ht.select(block=hl.nd.array(ht3.entries))
# print("after mt = mt.select(block=hl.nd.array(mt.entries))")
# print(ht.show())
# ht.describe()

In [ ]:
ht3 = ht3.annotate(W = hl.nd.array(ht3.W))

In [ ]:
# equation 20 preamble: split samples into k folds
k = 5
col_block_size = ht3.y.size // k

ht3.annotate_globals(folds = hl.nd.array(
    hl.range(1, k).map(lambd)
))

In [ ]:
# wnd = ht3.W.take(1)[0]

In [ ]:
wnd.shape

In [ ]:

ht2 = ht2.annotate(BMean = hl.nd.array(hl.map(lambda arr: hl.mean(arr), ht2.W)))
ht2.describe()

In [ ]:
ht2 = ht2.annotate(Wstd = hl.nd.array(hl.range(hl.len(ht2.W)).map(lambda i: hl.sum(ht2.W[i].map(lambda el: (el - ht2.WMean[i]) ** 2)) / (hl.len(ht2.W[i]) - 1)))
ht2 = ht2.annotate(Wnd = hl.nd.array(ht2.W))
ht2 = ht2.annotate(Wnormalized = (ht2.Wnd.T - ht2.WMean) / ht2.Wstd)

In [ ]:
np.array(ht2.W.take(1)[0][0]).shape

In [ ]:
ht2.Wnormalized.take(1)

In [ ]:
ht2.Wstd.take(1)

In [ ]:
ht2.WMean.take(1)

In [ ]:
ht2.Wnd.take(1)[0].T

In [ ]:
import time
start = time.time()
wstd = ht2.Wstd.collect()
wnd = ht2.Wnd.collect()
wmean = ht2.WMean.collect()
print("Took", time.time() - start)

In [ ]:
print(wstd[0].shape)
print("wnd", wnd[0].shape)
print("wmean", wmean[0].shape)

In [ ]:
wnd[0].T - np.array([1,2])

In [ ]:
np.array(ht2.W.take(1)[0]).shape

In [ ]:
ht2 = ht2.annotate(Wnd = hl.nd.array(ht2.W))
# You can't fold over NDArrayExpression, what in the actual fuck
# ht2 = ht2.annotate(Wmean = hl.fold(lambda i, j: i + j, 0.0, ht2.W)/ ht2.Wnd.size())
ht2 = ht2.annotate(Wmean = hl.mean(ht2.W))

In [ ]:
ht2 = ht2.annotate(WtW = ht2.Wnd.T @ ht2.Wnd)

In [ ]:
import time
start = time.time()
ht2.show()
print("Took", time.time() - start)

In [ ]:
list(alphas)

In [ ]:
ht.block.take(1)[0].shape  #1000 variatns by 100 samples
# shape Nsamples by 1
hl.eval(ht.Y.shape)

In [ ]:
# How regenie does it
# 1) stack = RidgeReducer()
# reduced_block_df = stack.fit_transform(block_df, 
#                              label_df, 
#                              sample_blocks, 
#                              covariates)


#out 
# Generated alphas: [  664469.6969697   877100.         1315650.         2631300.
#  65782500.       ]
# Out[12]: DataFrame[header: string, size: int, values: array<double>, header_block: string, sample_block: string, sort_key: int, mu: double, sig: double, alpha: string, label: string]


# 2)
# Calculate expected phenotypes per label and per sample
# with glow.wgr.linear_model.RidgeRegression under the leave-one-chromosome-out (LOCO) scheme.

# estimator = RidgeRegression()
# model_df, cv_df = estimator.fit(reduced_block_df, 
#                                 label_df, 
#                                 sample_blocks, 
#                                 covariates)
# model_df.cache()
# cv_df.cache()
# Generated alphas: [  6656.56565657   8786.66666667  13180.          26360.
#  659000.        ]
# Out[13]: DataFrame[label: string, alpha: string, alpha_value: double, r2_mean: double]


# Lets look at RidgeReducer
# the first function used is "fit" of ridge reducer
# that calls "apply_model": https://github.com/projectglow/glow/blob/054c3f4c714cf1693c964554842bf3ba67d76eb4/python/glow/wgr/linear_model/ridge_udfs.py#L266
# It says: This function takes a block X and a coefficient matrix B and performs the multiplication X*B.  The matrix resulting
 #   from this multiplication represents a block in a new, dimensionally-reduced block matrix.
# this doesn't make sense
# it's not dimensionally reduced I don't htink? It's just X*B. What is B here? We haven't solved for it yet.

# This calls cross_alphas_and_labels which does someting weird:
# In [3]: list(product((1,2,3), ("a","b","c")))                                                                                                          
# Out[3]: 
# [(1, 'a'),
#  (1, 'b'),
#  (1, 'c'),
#  (2, 'a'),
#  (2, 'b'),
#  (2, 'c'),
#  (3, 'a'),
#  (3, 'b'),
#  (3, 'c')]
# Ah, I guess this is just a convenient way of stacking; these are both strings, so it's 
# just comingin some labels that are strings, ok

# But wait, label is also used to indicate phenotypes here:
# label (phenotype). https://glow.readthedocs.io/en/latest/tertiary/whole-genome-regression.html?highlight=label#id8

# Ah from RidgeReducer documentation, 
# "When the RidgeReducer is initialized, it will assign names to the provided alphas and store them in a dictionary accessible as RidgeReducer.alphas."
# ah so the alpha labels are just keys...that's stupid, why the hell do they need names, indices work and are free

#

# @typechecked
# def cross_alphas_and_labels(alpha_names: Iterable[str], labeldf: pd.DataFrame,
#                             label: str) -> List[Tuple[str, str]]:
#     """
#     Crosses all label and alpha names. The output tuples appear in the same order as the output of
#     evaluate_coefficients.
#     Args:
#         alpha_names : List of string identifiers assigned to the values of alpha
#         labeldf : Pandas DataFrame of labels
#         label : Label used for this set of coefficients.  Can be 'all' if all labels were used.
#     Returns:
#         List of tuples of the form (alpha_name, label_name)
#     """
#     if label == 'all':
#         label_names = labeldf.columns
#     else:
#         label_names = [label]

#     return list(itertools.product(alpha_names, label_names))

# Wait, so if B is N * K (K is number of alphas)
# So This can only be multiplied against the pheontype matrix Y, right?
# XB would be N*M (N samples M Snps), or mayvbe M * N . Ok, so then M * N * N * K gives an M * K matrix
# 

In [ ]:
# The column field in a hail matrix table always? contains the sample id (can we have no samples?...would make sense yes, as long as the field could act as a column index)
#mt.sample_idx.show()

In [ ]:
mt2 = mtOrig.select_globals(Y = mtOrig.cols().index(mtOrig.cols(), all_matches=True).y)

In [ ]:
mtOrig(mtOrig.index(.y.show()

In [ ]:
lht = (mt
      .add_row_index(row_index)
      .select_rows()
      .select_cols()
      .localize_entries(entries))
# lht.describe()
# lht["locus"].show()
# mt.y.collect()
# mt2.key

# lht = lht.select_globals(FUCKYOU = 1)
# lht.describe()

In [ ]:
t = mt.localize_entries('entry_structs', 'columns')

In [ ]:
mt.select_cols("y").describe()

In [ ]:
# Current limitation ist hat dnd array doesnt support rectangles
# so number of samples in a group (columns) matches number of variants (rows)
# Not a real issue,
# block_size is the size in each dimension
darray = dnd.array(mt, "ac", block_size = 1000)
darray

In [ ]:
# it is blocked by some rows, and all of the columns
# so we get what we want for first pass...I think
# lets check with more samples
# Here we see that each block contains 1000 rows, and 100 columns, for 1000 variants, and all 100 samples
# (when n < block_size, n is used)
max(darray.m.r.collect())
# We have 10 blocks, since number of samples is smaller than block_size, and there are 10k variants
# print("N blocks:", len(dt))
# print("Shape:", dt[0].shape)

In [ ]:
darray.m.collect()

# Regenie method

In [ ]:
# From: https://glow.readthedocs.io/en/latest/tertiary/whole-genome-regression.html

# Step 1 import vcf, split multi allelics, mean impute genotypes, which means to replace missing values with the mean of non-missing values
from pyspark.sql.functions import col, lit

variants = spark.read.format('vcf').load(genotypes_vcf)
genotypes = glow.transform('split_multiallelics', variants) \
    .withColumn('values', glow.mean_substitute(glow.genotype_states(col('genotypes'))))

# Step 2 import phenotype data, and store it in a pandas dataframe called "label_df"
# also need to standardize phenotypes
# Note: this is only for continuous phenotypes in WGR
# Note: we can store these labels as entries,
# or for dnd arrays, as globals I believe
# Dimension: Y is N x 1, and in blocked form Bc X 1, where Bc is the column dim of the block
import pandas as pd

label_df = pd.read_csv(continuous_phenotypes_csv, index_col='sample_id')
label_df = ((label_df - label_df.mean())/label_df.std(ddof=0))[['Continuous_Trait_1', 'Continuous_Trait_2']]

# Step 3: import covaraites, and standardize
# Again, can store these as globals
# I believe the dimension here is N x C, where C is number of covariates
# except that if we block by samples as well as variants, this will need to be Bc X C, where Bc is the column dimension of the block
# This gets prepended to the X array, can do this using one of the numpy stacking operations
covariates = pd.read_csv(covariates_csv, index_col='sample_id')
covariates = (covariates - covariates.mean())/covariates.std(ddof=0)

# Step 4: Genotype matrix blocking
# This function creates an Br x Bc blocking of N * X
# Each block contains a certain number of rows an a certain number of columns
# They also remove rows that are completely uniform across all samples (since matrix will not be full rank if they don't)
# but I'm not sure how that affects things

# One advantage of WGR is that they allow rectangular blocks
# Description: 
# glow.wgr.functions.block_variants_and_samples creates two objects: a block genotype matrix and a sample block mapping.

# Parameters

# genotypes: Genotype DataFrame created by reading from any variant datasource supported by Glow, such as VCF. Must also include a column values containing a numeric representation of each genotype.
# sample_ids: List of sample IDs. Can be created by applying glow.wgr.functions.get_sample_ids to a genotype DataFrame.
# variants_per_block: Number of variants to include per block. We recommend 1000.
# sample_block_count: Number of sample blocks to create. We recommend 10.

# Return

# The function returns a block genotype matrix and a sample block mapping.

# Block genotype matrix

# If we imagine the block genotype matrix conceptually, we think of an NxM matrix X where each row n represents an individual sample, each column m represents a variant, and each cell (n, m) contains a genotype value for sample n at variant m. We then imagine laying a coarse grid on top of this matrix such that matrix cells within the same coarse grid cell are all assigned to the same block x. Each block x is indexed by a sample block ID (corresponding to a list of rows belonging to the block) and a header block ID (corresponding to a list of columns belonging to the block). The sample block IDs are generally just integers 0 through the number of sample blocks. The header block IDs are strings of the form ‘chr_C_block_B’, which refers to the Bth block on chromosome C. The Spark DataFrame representing this block matrix can be thought of as the transpose of each block xT all stacked one atop another. Each row represents the values from a particular column from X, for the samples corresponding to a particular sample block. The fields in the DataFrame are:

# header: A column name in the conceptual matrix X.
# size: The number of individuals in the sample block for the row.
# values: Genotype values for this header in this sample block. If the matrix is sparse, contains only non-zero values.
# header_block: An ID assigned to the block x containing this header.
# sample_block: An ID assigned to the block x containing the group of samples represented on this row.
# position: An integer assigned to this header that specifies the correct sort order for the headers in this block.
# mu: The mean of the genotype calls for this header.
# sig: The standard deviation of the genotype calls for this header.
# Sample block mapping

# The sample block mapping consists of key-value pairs, where each key is a sample block ID and each value is a list of sample IDs contained in that sample block.

# The order of these IDs match the order of the values arrays in the block genotype DataFrame.
# from glow.wgr.linear_model import RidgeReducer, RidgeRegression
# from glow.wgr.functions import block_variants_and_samples, get_sample_ids
# from pyspark.sql.functions import col, lit

# variants_per_block = 1000
# sample_block_count = 10
# sample_ids = get_sample_ids(genotypes)
# block_df, sample_blocks = block_variants_and_samples(
#     genotypes, sample_ids, variants_per_block, sample_block_count)


# # Step 5: dimensionality reduction
# # This runs RidgeReducer
# # This initializes some alphas
# reducer = RidgeReducer()


## Remaining questions:
# 1) how do we do the leave one out ? Will we need a mapping between blocks and chromosomes?

# Ah the paper has a much easier to follow thing.
# Algorithm 1 is: blcok the genotype matrices, then calcuate G.T * G and G.T * Y (or in our case G @ G.T and G @ Y since our G is M snps by N samples)

# Algorithm 2 seems like a way of getting around the sample blcoking thing
# They create gtg_sum and gty_sum, which are elementwise sums where for a given variant block, you sum all the results (but you do it once per sample i think)

In [ ]:
# ok, if too small then it blocks off the samples
# Ok, we have 200 rows, of block size 50, 50 * 200 == 10_000, number of variants
# columns we have 2 of, 2 * 50 == 100

# interestingly when I transpose, and then take M, I get wrong number of columns
max(darray2.T.m.c.collect())

In [ ]:
darray2.m = darray2.m.annotate(x = hl.nd.inv(darray2.m.block.T @ darray.m.block))

In [ ]:
darray2.m.show()

In [ ]:
# Read first MatrixTable and clean

# entries are now calls: An object that represents an individual’s call at a genomic locus
mt = hl.read_matrix_table("balding_nichols_3_100_1000.mt")

# don't understand meaning of this: returns the count of non-reference alleles from each call
mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 

mt.describe()

In [ ]:
# Read first MatrixTable and clean

# entries are now calls: An object that represents an individual’s call at a genomic locus
mt = hl.read_matrix_table("balding_nichols_3_100_1000.mt")

# don't understand meaning of this: returns the count of non-reference alleles from each call
mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 

mt.show()

In [ ]:
mt.describe()

In [ ]:
mt.n_partitions()

In [ ]:
wd = "/Users/alexkotlar/projects/hail/hail/python/test/data"
table = (hl.import_table(f"{wd}/1kg_annotations.txt", impute=True).key_by('Sample'))
table

In [ ]:
table.show()

In [ ]:
print(mt.col.dtype)

In [ ]:
hl.utils.get_1kg('data/')


In [ ]:
hl.import_vcf('data/1kg.vcf.bgz').write('data/1kg.mt', overwrite=True)


In [ ]:
mt = hl.read_matrix_table('data/1kg.mt')


In [ ]:
mt.rows().select().show(5)


In [ ]:
mt.count_rows()

In [ ]:
rwd = "/Users/alexkotlar/projects/regenie/example"
covars = hl.import_table(f"{rwd}/covariates.txt", impute=True, delimiter=" ")
phenos = hl.import_table(f"{rwd}/phenotype.txt", impute=True, delimiter=" ")
covariates

In [ ]:
covariates.show()

In [ ]:
mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 

mt.describe()

In [ ]:
mt.show()

In [ ]:

# Turn MatrixTable into Table

ht = mt.localize_entries("ent", "sample")
ht.describe()

In [ ]:
ht.show()

In [ ]:
# Functions for operating with Tables of ndarrays in Hail (from Tim)

from hail.expr import Expression, ExpressionException, \
    expr_float64, expr_call, expr_any, expr_numeric, expr_array, \
    expr_locus, \
    analyze, check_entry_indexed, check_row_indexed, \
    matrix_table_source, table_source

# Only groups by rows, NOT COLUMNS
def matrix_table_to_table_of_ndarrays(field, group_size, tmp_path = '/tmp/nd_table.ht'):
    """

    The returned table has two fields: 'row_group_number' and 'ndarray'.

    Examples
    --------
    >>> ht = matrix_table_to_table_of_ndarrays(mt.GT.n_alt_alleles(), 100)

    Parameters
    ----------
    field
    group_size
    tmp_path

    Returns
    -------

    """
    mt = matrix_table_source('matrix_table_to_table_of_ndarrays/x', field)
    mt = mt.select_entries(x = field)
    ht = mt.localize_entries(entries_array_field_name='entries')
    # now ht.entries is an array of structs with one field, x

    # we'll also want to mean-impute/variance-normalize/etc here
    ht = ht.select(xs = ht.entries.map(lambda e: e['x']))
    # now ht.xs is an array of float64

    # now need to produce groups of G
    ht = ht.add_index()
    ht = ht.group_by(row_group_number=ht.idx // group_size) \
        .aggregate(ndarray=hl.nd.array(hl.agg.collect(ht.xs)))
    # may require a .T on ndarray

    return ht.checkpoint(tmp_path, overwrite=True)

def chunk_ndarray(a, group_size):
    """Chunks a NDarray along the first axis in chunks of `group_size`.
    Parameters
    ----------
    a
    group_size

    Returns
    -------

    """
    n_groups = a.shape[0] // group_size
    groups = []
    for i in range(a.shape[0] // group_size):
        start = i * group_size
        end = (i + 1) * group_size
        groups.append(a[start:end, :])
    return groups

In [ ]:
hl.balding_nichols_model?

In [ ]:
bnm_mt.describe()

In [ ]:
mt.rows().show()

In [ ]:
test = dnd.array(bnm_mt, "GT")

In [ ]:
test.show()

In [ ]:
mt.show()

In [ ]:
bnm_mt.GT.

In [ ]:
mt.show()

In [ ]:
mt.GT.show()

In [ ]:
df = hl.utils.range_table(100)
df.show()

In [ ]:
n_partitions = mt.n_partitions()
block_size = 4096

In [ ]:
import json
fast_codec_spec = json.dumps({
        "name": "BlockingBufferSpec",
        "blockSize": 64 * 1024,
        "child": {
            "name": "LZ4FastBlockBufferSpec",
            "blockSize": 64 * 1024,
            "child": {
                "name": "StreamBlockBufferSpec"}}})

In [ ]:
from hail.utils.java import Env

In [ ]:
n_rows, n_cols = mt.count()
n_block_rows = (n_rows + block_size - 1) // block_size
n_block_cols = (n_cols + block_size - 1) // block_size
entries, cols, row_index, col_blocks = (Env.get_uid() for _ in range(4))

In [ ]:
entries

In [ ]:
cols

In [ ]:
row_index

In [ ]:
n_block_rows

In [ ]:
n_block_cols

In [ ]:
np.array([[0,1,2], [3,4,5]])

In [ ]:
nd = np.array([[0,1,2], [2,3,4]])

n_rows = nd.shape[0]

In [ ]:
n_rows

In [ ]:
nd[3 // n_rows, 3 % n_rows]

In [ ]:
4 // n_rows

In [ ]:
4 % n_rows

In [ ]:

nd = np.array([[0,1,2], [3,4,5]])
nd

In [ ]:
nd[1,1]

In [ ]:
4 % 2

In [ ]:
4 // 2

In [ ]:
nd[4 % 2, 4 // 2]

In [ ]:
i = 1
n_rows = 2
n_cols = 3

nd.T[i % n_rows, i // n_cols]

In [ ]:
nd[i // n_cols, i % n_cols]

In [ ]:
4 // 3

In [ ]:
nd.shape

In [ ]:
10 % 3

In [ ]:
10 - 3 * (10 // 3)

In [ ]:
def eye(N, M=None, dtype=hl.tfloat64):
    """
    Construct a 2-D :class:`.NDArrayExpression` with ones on the *main* diagonal
    and zeros elsewhere.

    Parameters
    ----------
    N : :class:`.NumericExpression` or Python number
      Number of rows in the output.
    M : :class:`.NumericExpression` or Python number, optional
      Number of columns in the output. If None, defaults to `N`.
    dtype : numeric :class:`.HailType`, optional
      Element type of the returned array. Defaults to :py:data:`.tfloat64`

    Returns
    -------
    I : :class:`.NDArrayExpression` representing a Hail ndarray of shape (N,M)
      An ndarray whose elements are equal to one on the main diagonal, zeroes elsewhere.

    See Also
    --------
    :func:`.identity`
    :func:`.diagonal`

    Examples
    --------
    >>> hl.eval(hl.nd.eye(3))
    array([[1., 0., 0.],
           [0., 1., 0.],
           [0., 0., 1.]])
    >>> hl.eval(hl.nd.eye(2, 5, dtype=hl.tint32))
    array([[1, 0, 0, 0, 0],
           [0, 1, 0, 0, 0]], dtype=int32)
    """

    n_row = hl.int32(N)
    if M is None:
        n_col = n_row
    else:
        n_col = hl.int32(M)

    return hl.nd.array(hl.range(0, n_row * n_col).map(
        lambda i: hl.cond((i // n_col) == (i % n_col),
                          hl.literal(1, dtype),
                          hl.literal(0, dtype))
    )).reshape((n_row, n_col))

In [ ]:
for i in range(13):
    for y in range(13):
        assert(np.allclose(hl.eval(eye(i,y)), np.eye(i,y)))

In [ ]:
i = 5
i - (i // n_cols) * n_cols

In [ ]:
import numpy as np
import hail as hl
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])
ah = hl.nd.array([1,2,3])
bh = hl.nd.array([2,3,4])

ah2 = hl.nd.array([[1], [2], [3]])
bh2 = hl.nd.array([[2], [3], [4]])
print(ah.ndim)
print("stfuff")
# print(hl.eval(hl.nah._broadcast(2)))
print(hl.eval(hl.nd.vstack((ah,bh))))
print(hl.eval(hl.nd.vstack((ah2,bh2))))
print("past")
print(hl.eval(hl.nd.concatenate((ah,bh),axis=0)))
print(np.vstack((a,b)))
print(hl.eval(ah.shape == bh.shape))
# print(hl.eval(hl.nd.resha))
print(ah.shape)
print(hl.array((ah,bh)))

In [ ]:
import hail as hl
import numpy as np
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])

seq = (a,b)
assert(np.allclose(hl.eval(hl.nd.vstack(seq)), np.vstack(seq)))

a = np.array([[1], [2], [3]])
b = np.array([[2], [3], [4]])
seq = (a,b)
assert(np.allclose(hl.eval(hl.nd.vstack(seq)), np.vstack(seq)))

In [ ]:

def _broadcast(nd, n_output_dims, axis=0):
        assert nd.ndim < n_output_dims
        print("nd.ndim", nd.ndim)
        # Right-align existing dimensions and start prepending new ones
        # to the left: e.g. [0, 1] -> [3, 2, 0, 1]
        # Based off numpy broadcasting with the assumption that everything
        # can be thought to have an infinite number of 1-length dimensions
        # prepended
        old_dims = range(nd.ndim)
        if axis == 1:
            new_dims = range(nd.ndim, n_output_dims)
        else:
            new_dim = range(nd.ndim, n_output_dims)
        print("old dims", old_dims)
        print("new_dims", new_dims)
        idx_mapping = list(reversed(new_dims)) + list(old_dims)
        print(idx_mapping)

In [ ]:
_broadcast(x, 2)

In [ ]:
ah.ndim

In [ ]:
x = np.array([1, 2])
y = np.expand_dims(x, axis=1)
y.shape

In [ ]:
x = hl.nd.array([1., 2.])
y = hl.nd.array([3., 4.])
hl.eval(hl.nd.concatenate((x, y), axis=0))

In [ ]:
np.vstack(([1,2,3],[4,5,6]))

In [ ]:
a = np.array([[1],[2],[3]])
b = np.array([[2],[3],[4]])

np.hstack((a,b))

In [ ]:
hl.eval(hl.nd.vstack((a,b)))

In [ ]:
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])

seq = (a,b)
assert(np.array_equal(hl.eval(hl.nd.vstack(seq)), np.vstack(seq)))

In [ ]:
for i in range(13):
            for y in range(13):
                assert(np.array_equal(hl.eval(hl.nd.eye(i,y)), np.eye(i,y)))

In [ ]:
x = np.array([[1., 2.], [3., 4.]])
y = np.array([[5.], [6.]])
np_res = np.concatenate([x, y], axis=1)

res = hl.eval(hl.nd.concatenate([x, y], axis=1))
assert np.array_equal(np_res, res)

In [ ]:
c = np.random.randn(5, 5)
d = np.linalg.inv(c)
dhail = hl.eval(hl.nd.inv(c))
assert np.allclose(dhail, d)

In [ ]:
hl.eval(hl.nd.eye(2, 5, dtype=hl.tint32))

In [ ]:
x = hl.nd.array([[1., 2.], [3., 4.]])
y = hl.nd.array([[5.], [6.]])
res = hl.nd.concatenate([x, y], axis=1)
hl.eval(res)

In [ ]:
import hail as hl
import numpy as np
a = hl.nd.array([1, 2, 3])
b = hl.nd.array([2, 3, 4])
hl.eval(hl.nd.vstack((a,b)))

In [ ]:
a = hl.nd.array([[1.], [2], [3]])
b = hl.nd.array([[2.], [3], [4]])
hl.eval(hl.nd.vstack((a,b)))

In [ ]:
hl.eval(hl.nd.identity(3))

In [ ]:
# import glow
# import hail as hl
# import numpy as np
# from pyspark.sql import SparkSession
# from pyspark import SparkContext

# hl.stop()
# hl.init()
# spark = SparkSession(SparkContext._active_spark_context)
# glow.register(spark)
# spark = SparkSession.builder\
#     .config('spark.jars.packages', 'io.projectglow:glow_2.11:0.5.0')\
#     .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
#     .getOrCreate()
# glow.register(spark)



In [ ]:
# thanks to Eric Czech
# https://github.com/projectglow/glow/issues/257
# Ah, that wasn't the issue; it doesn't look like you can have hail and glow running
# at the same time, because of Spark
# also good discussion here: https://github.com/projectglow/glow/issues/255
import glow
from glow import linear_regression_gwas, expand_struct
import numpy as np
from pyspark.ml.linalg import DenseMatrix
from pyspark.sql.session import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F

spark = SparkSession.builder\
    .config('spark.jars.packages', 'io.projectglow:glow_2.11:0.5.0')\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .getOrCreate()
glow.register(spark)

In [ ]:
variants_path = 'dbfs:/databricks-datasets/genomics/gwas/hapgen-variants.delta'
phenotypes_path = '/dbfs/databricks-datasets/genomics/gwas/Ysim_test_simulation.csv'
covariates_path = '/dbfs/databricks-datasets/genomics/gwas/Covs_test_simulation.csv'

y_hat_path = '/dbfs/tmp/wgr_y_hat.csv'
gwas_results_path = '/dbfs/tmp/wgr_gwas_results.delta'

base_variant_df = spark.read.format('delta').load(variants_path)
variant_df = glow.transform('split_multiallelics', base_variant_df) \
  .withColumn('values', mean_substitute(genotype_states(col('genotypes')))) \
  .filter(size(array_distinct('values')) > 1)

In [ ]:
import glow
from glow import *
from glow.wgr.functions import *
from glow.wgr.linear_model import *

import numpy as np
import pandas as pd
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark
from matplotlib import pyplot as plt
from bioinfokit import visuz
import DBUtils as dbutils

# dbutils is a databricks-only thing
# pyspark.dbutils.widgets.text('variants_per_block', '1000')
# pyspark.dbutils.widgets.text('sample_block_count', '10')

# Trying to understand DNDArray

In [ ]:
# import json
# import numpy as np
# from typing import Optional, Tuple, Callable

# from hail.utils.java import Env
# from hail.utils import range_table, new_temp_file
# from hail.expr import Expression
# from hail import nd
# from hail.matrixtable import MatrixTable
# from hail.table import Table
# from hail.experimental.dnd import DNDArray
# import hail as hl





# block_size = 1000
# row_block_size = 1000
# col_block_size = 1000
# entrc_field = "ac"

# n_rows, n_cols = mt.count()
# n_block_rows = (n_rows + block_size - 1) // block_size
# n_block_cols = 1 #(n_cols + block_size - 1) // block_size
# print("n_cols", n_cols, "n_rows", n_rows, "n_block_rows", n_block_rows, "n_block_cols", n_block_cols)
# entries, cols, row_index, col_blocks = (Env.get_uid() for _ in range(4))


# ################### localize_entries takes the entry field, which is separated into n columns, and stuffs all of that data into a single field, which is a 
# # array of structs, where the struct contains all of the entry fields you specied (I suppose you could specify multiple? else struct is wasteful)
# # I suppose since it's an array, we retain the order found in the original mt, so sample_idx, or sample names (which I think is a column field?) should remain
# # There is something added called col_idx...not sure what that is
# mt = (mt
#       .select_globals()
#       .select_rows()
#       .select_cols()
#       .add_row_index(row_index)
#       .localize_entries(entries, cols))
# print("""After mt = (mt
#       .select_globals()
#       .select_rows()
#       .select_cols()
#       .add_row_index(row_index)
#       .localize_entries(entries, cols))""")
# print(mt.show())
# mt.describe()

# ################## This step transformed the array of structs to an array of the values of one of the fields in that struct ########
# # FIXME: remove when ndarray support structs
# mt = mt.annotate(**{entries: mt[entries][entrc_field]})
# print("after mt.annotate(**{entries: mt[entries][entrc_field]})")
# print(mt.show())
# mt.describe()

# ################## This step creates a new field, labeled whatever col_blocks stores
# # creating an array of struct{c: <int>, entries: <array from place to next blcok>}
# # We currently don't need this step, since we're not blocking on column
# mt = mt.annotate(
#     **{col_blocks: hl.range(n_block_cols).map(
#         lambda c: hl.struct(
#             c=c,
#             entries=mt[entries][(c * block_size):((c + 1) * block_size)]))}
# )
# print("""after mt = mt.annotate(
#     **{col_blocks: hl.range(n_block_cols).map(
#         lambda c: hl.struct(
#             c=c,
#             entries=mt[entries][(c * block_size):((c + 1) * block_size)]))}
# )""")
# print(mt.show())
# mt.describe()


# ################## This creates 2 columns, one labeled "c" the other "entries"
# # which are the fields of the struct we created above
# # again we don't need this

# mt = mt.explode(col_blocks)

# print("after mt = mt.explode(col_blocks)")
# print(mt.show())
# mt.describe()

# ################ This gets rid of the fields entries and col_blocks, which is kind of ugly magic
# mt = mt.select(row_index, **mt[col_blocks])
# print("after mt = mt.select(row_index, **mt[col_blocks])")
# print(mt.show())
# mt.describe()
# ################ My own test: For some reason this mt[row_index].collect() seems to just return all the row indices
# # but mt[row_index].show() shows the full matrix table...
# # anyway, this seems fairly clearly to be usable in table context to count the row
# #print("mt[row_index].collect()")
# #print(mt[row_index].collect())
# ################ This greates groupings, I think. row_index I'm not sure where it come from, 
# mt = mt.annotate(r=hl.int(mt[row_index] // block_size))
# print("after mt = mt.annotate(r=hl.int(mt[row_index] // block_size))")
# print(mt.show())
# mt.describe()

# mt = mt.key_by(mt.r, mt.c)
# print("after mt = mt.key_by(mt.r, mt.c)")
# print(mt.show())
# mt.describe()
# print("what type is it", mt)

# # Not sure why entries=hl.sorted is neededd here
# mt = mt.group_by(mt.r, mt.c).aggregate(
#     entries=hl.sorted(
#         hl.agg.collect(hl.struct(row_index=mt[row_index], entries=mt.entries)),
#         key=lambda x: x.row_index
#     ).map(lambda x: x.entries))
# print("""after mt = mt.group_by(mt.r, mt.c).aggregate(
#     entries=hl.sorted(
#         hl.agg.collect(hl.struct(row_index=mt[row_index], entries=mt.entries)),
#         key=lambda x: x.row_index
#     ).map(lambda x: x.entries))""")
# print(mt.show())
# mt.describe()

# mt = mt.select(block=hl.nd.array(mt.entries))
# print("after mt = mt.select(block=hl.nd.array(mt.entries))")
# print(mt.show())
# mt.describe()

# mt = mt.select_globals(
#     r_field='r',
#     c_field='c',
#     n_rows=n_rows,
#     n_cols=n_cols,
#     n_block_rows=n_block_rows,
#     n_block_cols=n_block_cols,
#     block_size=block_size)
# print("""after mt = mt.select_globals(
#     r_field='r',
#     c_field='c',
#     n_rows=n_rows,
#     n_cols=n_cols,
#     n_block_rows=n_block_rows,
#     n_block_cols=n_block_cols,
#     block_size=block_size)""")
# print(mt.show())
# mt.describe()

# fname = new_temp_file()
# mt = mt.key_by(mt.r, mt.c)
# print("after mt = mt.key_by(mt.r, mt.c)")
# print(mt.show())
# mt.describe()

# mt.write(fname, _codec_spec=DNDArray.fast_codec_spec)
# t = hl.read_table(fname, _intervals=[
#     hl.Interval(hl.Struct(r=i, c=j),
#                 hl.Struct(r=i, c=j + 1))
#     for i in range(n_block_rows)
#     for j in range(n_block_cols)])
# print("""table resulting from t = hl.read_table(fname, _intervals=[
#     hl.Interval(hl.Struct(r=i, c=j),
#                 hl.Struct(r=i, c=j + 1))
#     for i in range(n_block_rows)
#     for j in range(n_block_cols)])""")
# print(t.show())
# mt.describe()

# mtNew = mt